# Kaggle TPS 2021 Octboer

## Optuna Catboost Starter

### Check GPU

In [1]:
# Google Colab
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Oct  4 06:28:38 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Check Memory

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


### PIP Installs

In [6]:
# Next two lines are for Google Colab GPU and working with AutoGluon
!pip uninstall -y mkl
!pip install --upgrade mxnet-cu100

!pip install optuna
# Use the autogluon version of catboost.  Had better success using this with GPU than doing the install of Catboost
!pip install autogluon


Found existing installation: mkl 2019.0
Uninstalling mkl-2019.0:
  Successfully uninstalled mkl-2019.0
     |████████████████████████████████| 352.6 MB 14 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 352 kB 16.7 MB/s 
     |████████████████████████████████| 52 kB 1.9 MB/s 
     |████████████████████████████████| 273 kB 74.6 MB/s 
     |████████████████████████████████| 56 kB 6.1 MB/s 
     |████████████████████████████████| 22.3 MB 1.3 MB/s 
     |████████████████████████████████| 27.4 MB 57.4 MB/s 
     |████████████████████████████████| 786 kB 58.4 MB/s 
     |████████████████████████████████| 4.2 MB 65.2 MB/s 
     |████████████████████████████████| 131 kB 77.1 MB/s 
     |████████████████████████████████| 206 kB 78.3 MB/s 
     |████████████████████████████████| 1.3 MB 72.1 MB/s 
     |████████████████████████████████| 1

### Imports

In [7]:
import random
import os
from google.colab import output
import optuna
from pathlib import Path


import catboost as cb
from catboost import CatBoostRegressor, Pool, CatBoostClassifier

import pandas as pd

from sklearn.ensemble import StackingRegressor
from sklearn.model_selection import train_test_split, cross_val_score, KFold, StratifiedKFold
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
import pickle
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.metrics import roc_auc_score
import json


#importing all the plot functions
from optuna.visualization import plot_edf
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice


import lightgbm as lgb
import json

import pandas as pd

from sklearn.ensemble import StackingRegressor
from sklearn.model_selection import train_test_split, cross_val_score, KFold, StratifiedKFold
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
import pickle
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.metrics import roc_auc_score

#importing all the plot functions
from optuna.visualization import plot_edf
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice

import logging

## Google Drive Mount

* This is used to save models and output
* Note that for this Kaggle competition, the train.csv and test.csv are quite large and take a while to upload and store.

In [8]:
working_dir = 'MyDrive/kaggle/202110'
drive_dir = '/content/drive/'
model_dir = 'models'
output_dir = 'output'
data_dir = 'data'
from google.colab import drive
drive.mount(drive_dir, force_remount=True)  # Do this once only

mounted_drive = f'{drive_dir}/{working_dir}' #''
os.chdir(mounted_drive)


Mounted at /content/drive/


### Datasets

In [9]:
pd.set_option("display.max_rows", 999)
pd.set_option("display.max_columns", 999)


In [10]:
X = pd.read_csv(f'./{data_dir}/reduced_mem_imputed.csv')
test_X = pd.read_csv(f'./{data_dir}/reduced_mem_imputed_test.csv') # this is the Kaggle test dataset

In [11]:
y = X[['target']]
X = X.drop(columns = ['id', 'target'])

test_X = test_X.drop(columns = ['id'])

N_SPLITS = 5 # if you're using cross_score_validation
EARLY_STOPPING_ROUNDS = 300
SEED = 42

### Model tuning

Optuna 
* study - this is the tuning done over several iterations of different hyperparameters
* trial - this is the individual evaluation/trial of selected hyperparameters
* objective function - defines the logic that Optuna will use for a single trial
* callback - allows actions after the objective function has returned the ranking metric to Optuna.  

At this point Optuna does not save the best model for use after the study.  Instead it will return the best parameters of the study.  However the study can take a long time, so a callback is used to save the best model (as of yet) to a file.

**Note that there should be a sample file called best_{model_type} which will be replaced by the best model of the study.  Right now the job will err out, if the model isn't there. ** 

In [13]:
def kfold_objective(trial):
   
    
    # model training and evaluation
  

    
    #'task_type': 'GPU',
    #    'devices':'0:1',
    # 'reg_lambda': trial.suggest_int('reg_lambda', 30, 35),
    # while catboost has a depth max of 16, memory issues were seen.  Depth takes up a lot of memory, so lowering max to 8
    # 'subsample': trial.suggest_float('subsample', 0.5, 0.8),  
    # 'use_best_model' : True, - from notebook https://www.kaggle.com/manabendrarout/custom-stacking-of-classifiers-gpu-tps-sep2021/
    # 'depth' : 5, - from above    
    # 'learning_rate': trial.suggest_float('eta', 0.011, 0.5),
    # 'max_depth': trial.suggest_int('max_depth', 5,8),
    kfolds = StratifiedKFold(n_splits=N_SPLITS, shuffle=True)
    
    
    param = {
        'max_depth': 7,
        'learning_rate': trial.suggest_float('eta', 0.001, 0.2),
        'iterations': trial.suggest_int('iterations', 1000, 10000),
        'reg_lambda': trial.suggest_int('reg_lambda', 30, 35),
        'bootstrap_type': 'Bernoulli', 
        'eval_metric':'AUC',
        'subsample': trial.suggest_float('subsample', 0.5, 0.6),  
        'thread_count': 4,
        'task_type': 'GPU',
        'devices':'0:1',
        'verbose': False
    }
    # output: evaluation score
    
    print(f'STARTING Param: {param}')  # Printing this before the cross val runs, so we have an idea if this is a long running job
    auc_score = 0
    keep_model = cb.CatBoostClassifier(**param)
    auc = []

    for train_index, test_index in kfolds.split(X, y):
      
      X_train, X_test = X.iloc[train_index], X.iloc[test_index]
      y_train, y_test = y.iloc[train_index], y.iloc[test_index]
      eval_dataset = Pool(X_test, y_test)
      

      model = cb.CatBoostClassifier(**param, use_best_model=True)
      model.fit(X_train, y_train, eval_set=eval_dataset)
      y_eval_predproba = model.predict_proba(X_test)
      new_auc_score = roc_auc_score(y_test, y_eval_predproba[:,1])
      auc.append(float(new_auc_score))
      if auc_score < new_auc_score:
        auc_score = new_auc_score 
        keep_model = model

      #print(f'Fold AUC Score {new_auc_score}')    

    ##########################
    # After KFold processing
    # Record the results to a file for further analysis outside the notebook
    ##########################
    print(f'Trial: all scores {auc}')
    print(f'Trial: Best Roc AUC score {auc_score}')
    result = {}
    result['optuna_score'] = float(auc_score) # score that is returned for the optuna trial
    
    
      
    result['scores'] = auc
    result['params'] = dict(keep_model.get_all_params())
    print(keep_model.get_all_params())

    with open('./artifacts/catboost_results.json', 'a') as f:
        json.dump(dict(result), f)
        f.write(',\n')

    

    filename = f'./models/catboost_{trial.number}'
    
    pickle.dump(keep_model, open(filename, 'wb'))
    
    return auc_score
    
def kfold_callback(study, trial):
    # The callback is used to capture the best model.  It checks for the best trial number and renames the model for that trial
    # Persisting the best model, means that we can unpickle it and run further tests whle Optuna keeps iterating to find a better tuning
    filename = f'./models/catboost_{trial.number}'
    
    best_model_name = './models/best_catboost'
    if study.best_trial.number == trial.number:
        os.remove(best_model_name)
        os.rename(filename, best_model_name)
    else:
        os.remove(filename)
    print(f'Trial: {trial.number} -  filename {filename} best_model_name{best_model_name}  best_trial_number {study.best_trial.number}')
    print('------------------')

In [14]:
def plot_results(study):

  # Code below is from a blog post - need to dig up reference
  # To get the dictionary of parameter name and parameter values:
  print("Return a dictionary of parameter name and parameter values:",study.best_params)
  
  # To get the best observed value of the objective function:
  print("Return the best observed value of the objective function:",study.best_value)
  
  # To get the best trial:
  print("Return the best trial:",study.best_trial)
  
  # To get all trials:
  print("Return all the trials:", study.trials)

  # Visualize the optimization history. See :func:`~optuna.visualization.plot_optimization_history` for the details.
  plot_optimization_history(study)

In [ ]:
# Create the best model file if it doesn't exist, otherwise don't alter it
# This file gets updated with the best scoring model of the study
best_model_name = './models/best_catboost'
best_model_file = Path(best_model_name)
best_model_file.touch(exist_ok=True)

study = optuna.create_study(direction='maximize')
study.optimize(kfold_objective, n_trials=100, callbacks=[kfold_callback])

plot_results(study)


[I 2021-10-04 07:04:38,297] A new study created in memory with name: no-name-567bf633-54c4-4581-b9c6-29a0f0dc0c6a


Param: {'max_depth': 7, 'learning_rate': 0.10693487929452573, 'iterations': 9043, 'reg_lambda': 32, 'bootstrap_type': 'Bernoulli', 'eval_metric': 'AUC', 'subsample': 0.5153243657972296, 'thread_count': 4, 'task_type': 'GPU', 'devices': '0:1', 'verbose': False}


In [ ]:
# load model from file
def load_model(model_path):
  model_file = open(model_path, 'rb')
  colab_model_eval = pickle.load(model_file)
  model_file.close()
  return colab_model_eval


In [ ]:
best_model_name = './models/best_catboost'
colab_model_eval = load_model(best_model_name)
# You may need to fit the model before predicting results

In [ ]:
roc_auc_score(y, colab_model_eval.predict_proba(X)[:, 1])


In [ ]:

y_predproba = pd.DataFrame(colab_model_eval.predict_proba(test_X))
y_predproba['id'] = y_predproba.index + len(X)
y_predproba.to_csv('./artifacts/catboost_predproba.csv', index=False)
results_df = y_predproba.rename(columns={1:'claim'}).copy()
results_df = results_df[['id','claim']]
results_df.to_csv('./artifacts/catboost_submission.csv', index=False)